

<p>This notebook does the following. </p>
<p>Part 1: Scrapes the twitter and download the tweets mentioning the 'ChatGPT'. </p>
<p>Part 2: Build a sentiment analysis model and identify tweets as positive and negative. </p>


In [ ]:
pip install tweepy
pip install beautifulsoup4

<p> This code requires twitter API, and access token. </p>

In [ ]:
import tweepy
import json
from bs4 import BeautifulSoup

# add your Twitter API credentials here
consumer_key = 'your_consumer_key'
consumer_secret = 'your_consumer_secret'
access_token = 'your_access_token'
access_token_secret = 'your_access_token_secret'

# authenticate with the Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# create the API object
api = tweepy.API(auth)



save all tweets that mention ChatGPT in a .json file. 

In [ ]:
# search for tweets that mention ChatGPT
tweets = api.search(q="ChatGPT", lang="en", count=100)

# extract the text of each tweet
tweet_texts = [tweet.text for tweet in tweets]

# save the tweets to a JSON file
with open('tweets.json', 'w') as f:
    json.dump(tweet_texts, f)


<p> PArt Two - Sentiment Analysis</p>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

# load the tweets from the JSON file
with open('tweets.json', 'r') as f:
    tweets = json.load(f)


# set the maximum number of words to use in the tokenizer
max_words = 10000

# create the tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')

# fit the tokenizer on the tweets
tokenizer.fit_on_texts(tweets)

# convert the text to sequences of integers
sequences = tokenizer.texts_to_sequences(tweets)

# set the maximum length of the sequences
max_length = 100

# pad the sequences to a fixed length
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')


<p> You can use your own data or a tensorflow pretrained data to do the classification. I am using TensorFlow Hub's data. </p>

In [ ]:
# load the pre-trained model from TensorFlow Hub
model = keras.Sequential([
    keras.layers.Input(shape=(max_length,), dtype=tf.int32),
    keras.layers.Embedding(max_words, 16),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(1, activation='sigmoid')
])
model.load_weights('https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1')


<p> Finally, predict the sentiment of each tweet. </p>


In [ ]:
# predict the sentiment of each tweet
predictions = model.predict(padded_sequences)

# classify the tweets as positive or negative
sentiments = ['positive' if prediction >= 0.5 else 'negative' for prediction in predictions]

# print the results
for i in range(len(tweets)):
    print(f'{sentiments[i]}: {tweets[i]}')
